In [ ]:
#export
from local.test import *
from local.basics import *

In [ ]:
from local.notebook.showdoc import *

In [ ]:
#default_exp callback.tensorboard

# Tensorboard

> Integration with [tensorboard](https://www.tensorflow.org/tensorboard) 

First thing first, you need to install tensorboard with
```
pip install tensoarboard
```
Then launch tensorboard with
``` 
tensorboard --logdir=runs
```
in your terminal. You can change the logdir as long as it matches the `log_dir` you pass to `TensorBoardCallback` (default is `runs` in the working directory).

In [ ]:
#export
import tensorboard
from torch.utils.tensorboard import SummaryWriter

In [ ]:
#export
class TensorBoardCallback(Callback):
    "Saves model topology, losses & metrics"
    # Record if watch has been called previously (even in another instance)
   
    def __init__(self, log_dir=None): store_attr(self, 'log_dir')

    def begin_fit(self):
        self.writer = SummaryWriter(log_dir=self.log_dir)

    def after_batch(self):
        #if self.train_iter==1: self.writer.add_graph(self.model, *self.xb)
        self.writer.add_scalar('train_loss', self.smooth_loss, self.train_iter)
        for i,h in enumerate(self.opt.hypers):
            for k,v in h.items(): self.writer.add_scalar(f'{k}_{i}', v, self.train_iter)

    def after_fit(self): self.writer.close()

## Test

In [ ]:
from local.vision.all import *
from local.callback.all import *

In [ ]:
pets = DataBlock(blocks=(ImageBlock, CategoryBlock), 
                 get_items=get_image_files, 
                 splitter=RandomSplitter(),
                 get_y=RegexLabeller(pat = r'/([^/]+)_\d+.jpg$'))

In [ ]:
dbunch = pets.databunch(untar_data(URLs.PETS)/"images", item_tfms=RandomResizedCrop(460, min_scale=0.75), bs=32,
                        batch_tfms=[*aug_transforms(size=299, max_warp=0), Normalize(*imagenet_stats)])

In [ ]:
opt_func = partial(Adam, lr=slice(3e-3), wd=0.01, eps=1e-8)

In [ ]:
learn = cnn_learner(dbunch, resnet50, opt_func=opt_func, metrics=error_rate).to_fp16()

In [ ]:
learn.fit_one_cycle(1, cbs=TensorBoardCallback(Path.home()/'tmp'/'runs'))

epoch,train_loss,valid_loss,error_rate,time
0,0.517636,0.263508,0.082544,00:34
